# Assignment 3

## Assumption
1. Rank of A is n

Implement the simplex algorithm to maximize the objective function, You need to implement the method discussed in class.

Input: CSV file with m+2 rows and n+1 column.
             The first row excluding the last element is the initial feasible point z of length n
             The second row excluding the last element is the cost vector c of length n
             The last column excluding the top two elements is the constraint vector b of length m
             Rows third to m+2 and column one to n is the matrix A of size m*n

Output: You need to print the sequence of vertices visited and the value of the objective function at that vertex


## Used Functions


## imports
**Details of Libraries:**
   - **`numpy`** - Importing for the numerical computations, like matrix operations & vector manipulations.
   - **`matplotlib.pyplot`**: Importing for the creation of plots that helps in visualizing results.
   - **`scipy.linalg.null_space`**: Importing for the computing of null space of an matrix, useful in the linear programming & feasibility checks.
   - **`seaborn`**: The visualization library built on top of `matplotlib`, will be used here for enhancing aesthetic quality of the plots.
   - **`csv`**: For reading & writing CSV files containing input data.
   - **`warnings`**: For controlling & manage warnings during program execution

In [1]:
## imports
import csv
import numpy as np
from scipy.linalg import null_space
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [2]:
# Suppress all warnings
warnings.filterwarnings("ignore")

np.random.seed(42)

## Reading CSV file
This `read_csv` function will read i/p CSV file & extracts required components for simplex algorithm. It will open file & convert data from that file into structured NumPy arrays-  first row going for parsing as initial feasible solution, second row as cost vector, &  an last column of remaining rows as right-hand side vector. Preceding columns from those rows form constraint matrix. This function makes sure that data is correctly structured & also ready for  the use in solving linear programming problem and also streamlining input process & minimizing manual errors.


In [3]:
def read_csv(filename):
    # to read CSV file (input) and extract init_feasible_sol, cost_vec, A, and rhs_vec matrix
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)

    # Extracting the initial feasible solution from 1st row
    init_feasible_sol = np.array([float(x) for x in data[0][:-1]])

    # Extracting the cost vector from the second row
    cost_vec = np.array([float(x) for x in data[1][:-1]])

    # Extracting the right-hand side (RHS) vector from  remaining rows
    rhs_vec = np.array([float(row[-1]) for row in data[2:]])

    # Extracting constraints matrix
    A = np.array([[float(x) for x in row[:-1]] for row in data[2:]])

    return A, init_feasible_sol, rhs_vec, cost_vec

## Reading & Visualizing Input Data

This code is reading CSV input file using `read_csv` function for extrating necessary components for simplex algorithm, such as constraint matrix \the \), initial feasible solution, cost vectors, & right-hand side vector. It will print the matrices to console for verification.

In [4]:
# to read the CSV file (input)
A, init_feasible_sol, rhs_vec, cost_vec = read_csv('testcase_3.csv')

# Printing the matrices
print("Constraint Matrix (A):\n", A)
print("\nInitial Feasible Solution (init_feasible_sol):", init_feasible_sol)
print("\nCost Vector (cost_vec):", cost_vec)
print("\nRight-Hand Side Vector (rhs_vec):", rhs_vec)

Constraint Matrix (A):
 [[1. 1.]
 [1. 1.]
 [1. 0.]]

Initial Feasible Solution (init_feasible_sol): [0. 0.]

Cost Vector (cost_vec): [3. 3.]

Right-Hand Side Vector (rhs_vec): [3. 3. 2.]


##  Identifying Tight Rows in Constraint Matrix

`find_tight_rows` function identifies which constraints in matrix \( A \) are tight for given initial feasible solution. It perform this by computing the diff between result of dot product of \( A \) & initial solution, & right-hand side vector \( b \). If an absolute difference is smaller than the specified tolerance (\( \epsilon \)), then that row is considered tight. Function then separates matrix \( A \) into 2  part-: `tight_rows`, which will be containing rows where constraints are tight, & `untight_rows`, which contains rows where constraints are not tight. This distinction is vey important to determinine which constraints are active in that feasible region of solution.


In [5]:
# determining tight rows from constraint matrix
def find_tight_rows(A, init_feasible_sol, rhs_vec, epsilon=1e-8):

    # Checking if the diff is less than a small epsilon to identify tight rows
    tight_mask = np.abs(np.dot(A, init_feasible_sol) - rhs_vec) < epsilon

    # Selecting the rows where the constraint is tight
    tight_rows = A[tight_mask]

    # Selecting the rows where the constraint are not tight
    untight_rows = A[~tight_mask]

    return tight_mask, tight_rows, untight_rows

## Identifying Tight & Untight Constraints for Initial Feasible Solution
This code calls `find_tight_rows` function for determining  which constraints are tight for that given initial feasible solution. Function returns the mask which is indicating tightness of each constraint, as well as specific rows in constraint matrix \( A \) that are tight (`tight_rows`) & those that are not found tight (`untight_rows`). This separation is important for the understanding of which constraints actively define feasible region of current solution in our simplex algorithm.


In [6]:
# Find tight rows for initial feasible solution
mask, tight_rows, untight_rows = find_tight_rows(A, init_feasible_sol, rhs_vec)

### Computing Directions Using Tight Constraints

The `compute_directions` function computes direction vectors based on the tight constraints of the current solution. It takes the matrix of tight constraints as input, where each row represents a constraint that is active for the given solution. The function attempts to calculate the inverse of the tight rows matrix and returns its negative value. This inverse matrix is used to determine the direction of movement within the feasible region. If the matrix is singular (non-invertible), an exception is raised, and the function prints an error message, returning `None` to indicate that the direction could not be computed. This function is crucial for the simplex algorithm, as it helps identify the direction in which the solution can move to improve the objective function.


In [7]:
# This fuction is for computing vectors using the tight rows.
def compute_directions(tight_rows):
    #  It is taking matrix of tight constraints as input.
    # Each row is representing the constraints that are currently active for given solution

    try:
        return -np.linalg.inv(tight_rows) # This is computing & returning - ve inverse of tight rows matrix.
    except np.linalg.LinAlgError:
        # If an tight_rows matrix is singular (i.e. non invertible) then the exception is raised.
        print("It is Singular Matrix, So cannot compute directions.")   # Handling non invertible matrix
        return None # Returning None for indicating that the directions could not be computed

## Checking Degeneracy

This function is checking for degeneracy in the linear programming problem by comparing no. of tight constraints (that is rows) to rank of matrix which is formed by those constraints.

In [8]:
def check_degeneracy(tight):
    # Checking if it is degenerate occurs. Degeneracy occurs if rows > columns
    return tight.shape[0] > tight.shape[1]

## Making a System Non-Degenerate

The `make_non_degenerate` function adjusts a given vector to ensure it is non-degenerate by introducing small perturbations based on a diminishing epsilon factor. In this case, it modifies each element of the input vector (`original_b`) by adding progressively smaller adjustments, making the system less likely to have degenerate constraints.

The steps involved in the algorithm are as follows -

1) Initialize Epsilon Reduction: Reduce the input epsilon value by the specified `factor` (default is 0.5).

2) Iterative Adjustment: For each element in `original_b`, compute an adjustment as $epsilon^{(i+1)}$, where i is the index of the element. Add this adjustment to the original value and store the result.

3) Return Adjusted Vector: Convert the adjusted values to a numpy array and return it along with the updated epsilon.

The function outputs the updated vector with adjusted values and the reduced epsilon value.

In [9]:
def make_non_degenerate(original_b, epsilon, factor=0.5):
    # Reducing epsilon by the factor
    epsilon = epsilon * factor

    # Creating a list to store updated values
    updated_b = []

    # Update each value in original_b
    for i in range(len(original_b)):

        adjustment = epsilon ** (i + 1)

        new_value = original_b[i] + adjustment

        # Storing the updated value
        updated_b.append(new_value)

    # Convert the updated values to a numpy array
    updated_b_array = np.array(updated_b)

    return updated_b_array, epsilon

## Transitioning to a Feasible Vertex

The `feasible_to_vertex` function identifies a feasible vertex from a given initial feasible solution, ensuring it satisfies the constraints and lies at a vertex of the feasible region. This function iteratively moves from an initial feasible solution to a vertex while satisfying constraints until a solution with full rank is found. It handles degeneracy by checking the validity of the tight constraints at each step.

The steps involved in the algorithm are as follows -

1) Initialization: Store the initial feasible solution (`init_feasible_sol`) and compute its cost. Check for the tight and untight constraints for the initial solution using `find_tight_rows`.

2) Check Vertex Condition: Compute the rank of the tight constraints. If the rank equals the problem dimension (n), the current solution is a vertex, and the function ends successfully.

3) Iterative Search for a Vertex: If the initial solution is not a vertex, iterate to find a valid vertex with the following steps -
+ Compute a direction vector (`u`) using the null space of the tight constraints.
+ Determine step sizes (`alphas`) to move along the direction vector while maintaining feasibility.
+ Select the smallest positive step size and update the solution.
+ Check for tight and untight constraints again, and update the rank.

4) Degeneracy Check: If degeneracy is present (`check_degeneracy`), return early to handle it externally.

The function outputs the feasible vertex found, values of cost function at visited points, and list of vertices (feasible points) visited during the optimization process or returns `None` if degenracy is detected.

In [10]:
def feasible_to_vertex(A, rhs_vec, init_feasible_sol, cost_vec, n):
    # array for storing visited vertices and respective costs
    vertices = []
    costs = []

    # Adding the the initial feasible vertex and cost to the array
    vertices.append(init_feasible_sol)
    costs.append(np.dot(cost_vec, init_feasible_sol))  # Calculating the initial cost

    # now setting the current solution to the initial feasible solution
    track_curr_feasible = init_feasible_sol
    iteration = 0

    # Finding tight and untight rows for the initial feasible solution
    mask, tight_rows, untight_rows = find_tight_rows(A, init_feasible_sol, rhs_vec)

    # computing rank of the tight rows
    if len(tight_rows) != 0:
        rank = np.linalg.matrix_rank(tight_rows)  # Number of independent constraints
    else:
        rank = 0

    # determining is initial feasible point is a vertex
    if rank != n:
        print("Since this feasible point is not a vertex, searching for a vertex.\n")
    else:
        # If it is a vertex return the current solution
        return track_curr_feasible, costs, vertices

    # Iterate until we find a feasible vertex
    while rank != n:
        iteration += 1

        # Avoiding infinite loops (terminate after too many iterations)
        if iteration > 10000:
            break

        # computing a direction vector
        if len(tight_rows) != 0:
            null_space_matrix = null_space(tight_rows)
            u = null_space_matrix[:, 0]
        else:

            u = np.random.rand(untight_rows.shape[-1])

        # Find the step size (alpha) to move along the direction vector u
        while True:
            alphas = [
                (_b_i - np.dot(a2_i, track_curr_feasible)) / np.dot(a2_i, u)
                for _b_i, a2_i in zip(rhs_vec[~mask], untight_rows)
            ]

            all_alphas = [alpha for alpha in alphas if alpha > 0]

            if len(all_alphas) == 0:

                u = u * (-1)
            else:
                break

        # Moving to the new feasible point using the smallest valid step size
        alpha = min(all_alphas)
        new_ver_opti = track_curr_feasible + alpha * u

        # Updating tight and untight rows for the new point
        mask, tight_rows, untight_rows = find_tight_rows(A, new_ver_opti, rhs_vec)
        track_curr_feasible = new_ver_opti  # Updating the current feasible solution

        # Updating the rank of the tight rows
        if len(tight_rows) != 0:
            rank = np.linalg.matrix_rank(tight_rows)
        else:
            rank = 0

        # Recording the new vertex and its cost
        costs.append(np.dot(cost_vec, new_ver_opti))
        vertices.append(new_ver_opti)

    # Checking for degeneracy
    if not check_degeneracy(tight_rows):
        return new_ver_opti, costs, vertices
    else:
        return (None,)

## Vertex-to-Vertex Optimization Using the Simplex Method

The `vertex_to_vertex` function performs an iterative optimization process using the Simplex method, moving between feasible vertices to improve the objective function.

The steps involved in the algorithm are as follows -

1) Initialization: Start with an initial feasible solution (`init_feasible_sol`) and compute its cost. Store visited vertices and their costs.

2) Finding Tight Constraints: Use `find_tight_rows` to identify tight and untight constraints at the current solution.

3) Checking Degeneracy: If degeneracy is detected via `check_degeneracy`, the process terminates.

4) Compute Directions: Calculate possible directions for movement (`compute_directions`). Select directions improving the objective function (positive dot product with `cost_vec`).

5) Step Size and Update: Compute step sizes for untight constraints and filter for positive values. If no valid step size exists, the problem is unbounded. Otherwise, move to the next vertex using the smallest positive step size and update the solution.

6) Termination: Repeat until no improving direction exists, indicating the optimal solution is reached.

The algorithm outputs the optimal solution, the cost function values at each visited vertex and the sequence of vistied vertices.

In [11]:
def vertex_to_vertex(A, rhs_vec, init_feasible_sol, cost_vec):
    # Lists to store visited vertices and their costs
    vertices = []
    costs = []

    # Starting with the initial feasible solution
    track_curr_feasible = init_feasible_sol
    new_ver_opti = track_curr_feasible

    # Adding the cost of the initial solution
    costs.append(np.dot(cost_vec, track_curr_feasible))
    vertices.append(track_curr_feasible)

    while True:
        # Identifing which rowsare tight at the current feasible solution
        mask, tight_rows, untight_rows = find_tight_rows(A, track_curr_feasible, rhs_vec)

        # Check for degeneracy
        if check_degeneracy(tight_rows):
            return (None,)

        # Compute possible directions
        directions = compute_directions(tight_rows).T
        pos_directions = []

        for direction in directions:
            if np.dot(direction, cost_vec) > 0:
                pos_directions.append(direction)

        if not pos_directions:
            return new_ver_opti, costs, vertices

        # first positive direction
        u = pos_directions[0]
        alphas = []

        # step sizes for all untight constraints
        for b_i, a2_i in zip(rhs_vec[~mask], untight_rows):
            numerator = b_i - np.dot(a2_i, track_curr_feasible)  # Remaining margin
            denominator = np.dot(a2_i, u)  # Impact of direction on constraint
            alphas.append(numerator / denominator)

        # Keep only positive step sizes
        positive_alphas = [alpha for alpha in alphas if alpha > 0]

        # the problem is unbounded ,If no valid step size exists,
        if len(positive_alphas) == 0:
            print("The problem is unbounded. An optimal solution cannot be found.")
            return None, costs, vertices

        # Moving to the new vertex using the smallest positive step size
        alpha = min(positive_alphas)
        new_ver_opti = track_curr_feasible + alpha * u
        track_curr_feasible = new_ver_opti

        # storing the cost and vertex
        costs.append(np.dot(cost_vec, new_ver_opti))
        vertices.append(new_ver_opti)

## Main Optimization Loop

The main function runs the process of solving a linear programming problem using a two-stage approach - moving to a feasible vertex and then optimizing through vertex-to-vertex traversal. It also makes sure that the problem remains non-degenerate through iterative adjustments. Here, it integrates the `feasible_to_vertex`, `vertex_to_vertex`, and `make_non_degenerate` functions. It first finds a feasible vertex and then optimizes the solution iteratively.

The steps involved in the algorithm are as follows -

1) Initialization: Set the input variables, such as the constraint matrix (`matrix_A`), original RHS vector (`vector_b_original`), initial feasible solution (`vector_z`), and cost vector (`vector_c`). Initialize epsilon for changes and an attempt counter.

2) Iterative Adjustments: If degeneracy is detected, modify the RHS vector (`vector_b`) using the `make_non_degenerate` function with continuously reduced epsilon values.

3) Feasible Vertex Search: Call `feasible_to_vertex` to find a feasible vertex. If unsuccessful, increment the attempt counter and restart.

4) Vertex-to-Vertex Optimization: Use the `vertex_to_vertex` function to optimize the objective by traversing between feasible vertices. If unsuccessful, adjust for degeneracy and retry.

The loop ends once both a feasible vertex and an optimal solution are found.

The function outputs the optimal solution, the values of cost function during vertex-to-vertex optimization and list of all vertices visited.

In [ ]:
# Main Optimization Loop

m, n = len(rhs_vec), len(cost_vec)
matrix_A = A
vector_b_original = rhs_vec
vector_z = init_feasible_sol
vector_c = cost_vec
dimension_n = n

epsilon = 0.1
attempt = 0
vector_b = vector_b_original

while True:
    if attempt > 0:
        vector_b, epsilon = make_non_degenerate(vector_b_original, epsilon)

    outputs1 = feasible_to_vertex(matrix_A, vector_b, vector_z, vector_c, dimension_n)
    if len(outputs1) == 1:
        attempt += 1
        continue

    new_ver_opti, feas2vert_z_all_cost, feas2vert_z_all = outputs1

    outputs2 = vertex_to_vertex(matrix_A, vector_b, new_ver_opti, vector_c)
    if len(outputs2) == 1:
        attempt += 1
        continue

    z_optimal, vert2vert_z_all_cost, vert2vert_z_all = outputs2
    break

Since this feasible point is not a vertex, searching for a vertex.

Since this feasible point is not a vertex, searching for a vertex.



## Printing Results


Print the sequence of visited vertices and their corresponding objective function costs during the feasible-to-vertex phase.

Print the initial vertex obtained after transitioning from a feasible point.

Print the optimal vertex and its cost after vertex-to-vertex optimization. Handle cases where the problem is unbounded.

In [ ]:
print("Feasible point to vertex:\n")
print("Sequence of visited Vertices are as follows:")

# Loop through the list of feasible points and their costs
for index in range(len(feas2vert_z_all)):
    print(f"Vertex {index + 1}: {feas2vert_z_all[index]}, Cost {index + 1}: {feas2vert_z_all_cost[index]}")

print("\nInitial vertex: ", new_ver_opti)

print("\nOptimal vertex is as follows:")
print(f"Point: {vert2vert_z_all[0]}")
print(f"Cost: {vert2vert_z_all_cost[0]}")

if np.all(z_optimal == None):
    print("\nThe problem is unbounded!")
else:
    print("\nOptimal vertex:")
    print(z_optimal)
